You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [14]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
from cryptography.utils import CryptographyDeprecationWarning

warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os
from dotenv import load_dotenv
import httpx
import openai

# Load environment variables
load_dotenv()

# Get the OpenAI API key from the environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Define your proxy settings
proxies = {
    "http://": "http://your_proxy_address:port",
    "https://": "http://your_proxy_address:port",
}

# Create an HTTP client with the proxy settings
http_client = httpx.Client(proxies=proxies)

# Initialize the OpenAI client with the HTTP client
openai_client = openai.OpenAI(api_key=openai_api_key, http_client=http_client)

# Example API call
response = openai_client.Completion.create(
    model=os.getenv("OPENAI_MODEL_NAME"),
    prompt="Once upon a time...",
    max_tokens=50
)

print(response.choices[0].text)


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [13]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


Exception while exporting Span batch.
Traceback (most recent call last):
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 773, in urlopen
    self._prepare_proxy(conn)
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1042, in _prepare_proxy
    conn.connect()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 721, in connect
    self._tunnel()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 259, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 403 Forbidden

The above exception was the direct cause of the following exception:

urllib3.exceptions.ProxyError: ('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden'))

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  



# Agent: Content Planner
## Final Answer: 
# Comprehensive Content Plan on Artificial Intelligence

## 1. Latest Trends in Artificial Intelligence

- **Generative AI**: The rise of models like ChatGPT and DALL-E that create content, images, and music based on prompts.
- **AI Ethics and Governance**: Growing discussions around the ethical implications of AI and the establishment of regulatory frameworks.
- **AI in Healthcare**: Utilization of AI for predictive analytics, personalized medicine, and robotic surgeries.
- **AI in Autonomous Systems**: Advances in self-driving vehicles and drones, highlighting real-world applications and breakthroughs.
- **Natural Language Processing (NLP)**: Continued improvements in conversational AI and voice-activated systems.
- **AI-powered Automation**: Businesses increasingly leveraging AI for process automation, increasing efficiency and reducing costs.

## 2. Key Players in Artificial Intelligence

- **Google DeepMind**: Pioneering work in neural 

Exception while exporting Span batch.
Traceback (most recent call last):
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 773, in urlopen
    self._prepare_proxy(conn)
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1042, in _prepare_proxy
    conn.connect()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 721, in connect
    self._tunnel()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 259, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 403 Forbidden

The above exception was the direct cause of the following exception:

urllib3.exceptions.ProxyError: ('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden'))

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  



# Agent: Content Writer
## Final Answer: 
```markdown
# Embracing the Future: A Comprehensive Look at Artificial Intelligence

## Introduction
Artificial Intelligence (AI) is no longer just a buzzword; it's a transformative force permeating every facet of our lives. As we advance deeper into the 21st century, AI is reshaping industries, enhancing our daily experiences, and pushing the boundaries of what we believe is possible. In fact, the global AI market is projected to reach over $390 billion by 2025, showcasing its growing importance and versatility.

As we explore the rapidly evolving landscape of AI, it's imperative to recognize the trends and challenges that come hand-in-hand with this technology. From generative AI to ethical governance, understanding AI is not just for the tech-savvy but for everyone invested in our shared future.

## Section 1: Trends Shaping the Future of AI
The advent of generative AI, exemplified by models like ChatGPT and DALL-E, marks a significant tre

Exception while exporting Span batch.
Traceback (most recent call last):
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 773, in urlopen
    self._prepare_proxy(conn)
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1042, in _prepare_proxy
    conn.connect()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 721, in connect
    self._tunnel()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 259, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 403 Forbidden

The above exception was the direct cause of the following exception:

urllib3.exceptions.ProxyError: ('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden'))

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  



# Agent: Editor
## Final Answer: 
```markdown
# Embracing the Future: A Comprehensive Look at Artificial Intelligence

## Introduction
Artificial Intelligence (AI) is no longer just a buzzword; it is a transformative force that permeates every facet of our lives. As we advance deeper into the 21st century, AI continues to reshape industries, enhance our daily experiences, and push the boundaries of what we believe is possible. In fact, the global AI market is projected to reach over $390 billion by 2025, underscoring its growing importance and versatility.

As we explore the rapidly evolving landscape of AI, it is imperative to recognize the trends and challenges that come hand in hand with this technology. From generative AI to ethical governance, understanding AI is not just for the tech-savvy but for everyone invested in our shared future.

## Section 1: Trends Shaping the Future of AI
The advent of generative AI, exemplified by models like ChatGPT and DALL-E, marks a significant 

- Display the results of your execution as markdown in the notebook.

In [20]:
from IPython.display import Markdown
Markdown(result)

# Unleashing the Power of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has become a ubiquitous term in today's technological landscape, encompassing a wide range of applications and advancements that are shaping the future of various industries. AI, in its essence, refers to the simulation of human intelligence processes by machines, particularly computer systems. The importance and relevance of AI in today's world cannot be overstated, as it has the potential to revolutionize how we live, work, and interact with technology. The latest trends in AI are paving the way for unprecedented developments in machine learning, natural language processing, and computer vision, among other areas.

## Latest Trends in Artificial Intelligence

### Machine Learning

Machine learning, a subset of AI, focuses on the development of algorithms that enable computers to learn from data and make predictions or decisions without being explicitly programmed. This technology has seen significant advancements in recent years, with applications ranging from predictive analytics in healthcare to algorithmic trading in finance. Machine learning is driving innovation across industries, empowering organizations to leverage data-driven insights for strategic decision-making.

### Natural Language Processing

Natural Language Processing (NLP) is another key area of AI that is revolutionizing how machines interact with and understand human language. The development of AI-powered language models, such as OpenAI's GPT-3, has opened new possibilities for applications in virtual assistants, chatbots, and content generation. NLP is enabling more natural and intuitive communication between humans and machines, enhancing user experiences and streamlining business processes.

### Computer Vision

Computer vision technology focuses on enabling machines to interpret and analyze visual information from the real world. Recent progress in computer vision has led to breakthroughs in areas like autonomous vehicles and surveillance systems. AI-powered vision systems are enhancing safety, efficiency, and decision-making capabilities in various domains, from transportation to security. The potential applications of computer vision are vast, promising a future where machines can perceive and understand the visual world like never before.

## Key Players in the AI Industry

### Tech Giants

Leading tech companies like Google, Amazon, and Microsoft are at the forefront of AI research and development, driving innovation in areas like machine learning, NLP, and computer vision. These industry giants are investing heavily in AI initiatives to stay competitive and deliver cutting-edge solutions to global markets. Google's DeepMind, Amazon's Alexa, and Microsoft's Azure AI are just a few examples of the groundbreaking AI projects led by these tech titans.

### Startups and Scaleups

In addition to established players, a growing number of startups and scaleups are entering the AI space, bringing fresh ideas and disruptive technologies to the market. These emerging companies are challenging traditional business models and pushing the boundaries of AI innovation. Startups like OpenAI, UiPath, and DataRobot are making waves with their AI-powered solutions, attracting investors and customers alike with their unique value propositions.

### Research Institutions and Universities

Research institutions and universities play a vital role in advancing AI through cutting-edge research and academic collaborations. Leading academic institutions like Stanford University, MIT, and Oxford University are at the forefront of AI research, contributing to breakthroughs in areas like deep learning, reinforcement learning, and robotics. These institutions serve as hubs of innovation and knowledge-sharing, fostering a vibrant ecosystem of AI talent and expertise.

## Noteworthy News in Artificial Intelligence

### Ethical Considerations

As AI technologies continue to advance, ethical considerations around AI ethics and responsible AI practices have come to the forefront of discussions. Issues like bias in AI algorithms, data privacy, and transparency in AI decision-making have raised concerns about the societal impact of AI. Stakeholders across industries are grappling with how to develop and deploy AI systems in an ethical and accountable manner, ensuring that AI benefits society as a whole while minimizing potential risks.

### Regulatory Updates

Governments around the world are responding to the rapid growth of AI with regulatory frameworks and policies to govern AI usage and deployment. From data protection laws to algorithmic accountability measures, regulations are being put in place to safeguard against misuse and abuse of AI technologies. Regulatory updates in AI aim to strike a balance between fostering innovation and protecting the rights and interests of individuals and communities affected by AI systems.

### Industry Collaborations

Partnerships and collaborations in the AI ecosystem are driving innovation and knowledge-sharing among industry players, research institutions, and government agencies. Cross-sector collaborations are enabling synergies and expertise exchange, leading to the development of new AI solutions and applications. Industry collaborations in AI are fostering a culture of cooperation and open innovation, accelerating the pace of AI advancement and adoption across diverse domains.

## Conclusion

In conclusion, Artificial Intelligence is a dynamic and rapidly evolving field that holds immense potential for transforming industries, driving innovation, and shaping the future of technology. The latest trends in AI, key players in the industry, and noteworthy news in AI underscore the multifaceted nature of AI and its far-reaching impact on society. As we navigate the complex landscape of AI advancements and challenges, staying informed and engaged with the latest developments in AI is essential for professionals, business owners, students, and researchers alike. By embracing the power of AI responsibly and ethically, we can unlock new opportunities and possibilities that will shape the future of AI-driven innovation.

*Stay updated on the latest trends and innovations in Artificial Intelligence by subscribing to our newsletter for exclusive insights and updates.*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [17]:
topic = "Best practices for using Crew AI to write patents"
result = crew.kickoff(inputs={"topic": topic})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Best practices for using Crew AI to write patents.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


ERROR:opentelemetry.sdk.trace.export:Exception while exporting Span batch.
Traceback (most recent call last):
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 773, in urlopen
    self._prepare_proxy(conn)
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1042, in _prepare_proxy
    conn.connect()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 721, in connect
    self._tunnel()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 259, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 403 Forbidden

The above exception was the direct cause of the following exception:

urllib3.exceptions.ProxyError: ('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden'))

The above exception was the direct cause of the following exception:





# Agent: Content Planner
## Final Answer: 
**Content Plan Document: Best Practices for Using Crew AI to Write Patents**

**1. Introduction**
   - **Purpose of the Article**: To provide insights and guidelines on utilizing Crew AI for patent writing effectively.
   - **Importance of the Topic**: With the rapid evolution of AI tools in intellectual property, it's essential for innovators and businesses to leverage these technologies to streamline the patent process.

**2. Target Audience Analysis**
   - **Primary Audience**: 
     - Patent attorneys and agents
     - Inventors and researchers
     - Businesses looking to protect their intellectual property
   - **Interests**:
     - Increasing efficiency in patent drafting
     - Understanding the capabilities of AI in legal documentation
     - Finding reliable tools for patent writing
   - **Pain Points**:
     - Time-consuming patent drafting processes
     - Understanding complex legal language
     - Keeping up with new AI technol

ERROR:opentelemetry.sdk.trace.export:Exception while exporting Span batch.
Traceback (most recent call last):
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 773, in urlopen
    self._prepare_proxy(conn)
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1042, in _prepare_proxy
    conn.connect()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 721, in connect
    self._tunnel()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 259, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 403 Forbidden

The above exception was the direct cause of the following exception:

urllib3.exceptions.ProxyError: ('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden'))

The above exception was the direct cause of the following exception:





# Agent: Content Writer
## Final Answer: 
```markdown
# Best Practices for Using Crew AI to Write Patents

## Introduction

In the fast-paced world of intellectual property, the emergence of artificial intelligence (AI) tools is revolutionizing the way patents are written. One standout entity in this realm is Crew AI, an innovative platform that empowers inventors, patent attorneys, and businesses to streamline their patent drafting processes. Understanding how to leverage Crew AI effectively can save time, reduce frustration, and improve the overall quality of patent applications. This article aims to delve into the best practices for using Crew AI in patent writing, ultimately aiding professionals in navigating the complexities of patent documentation.

The relevance of this topic cannot be overstated, given the growing reliance on AI technologies in legal processes. With platforms like Crew AI, the opportunity to enhance efficiency and accuracy in patent writing has never been cle

ERROR:opentelemetry.sdk.trace.export:Exception while exporting Span batch.
Traceback (most recent call last):
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 773, in urlopen
    self._prepare_proxy(conn)
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1042, in _prepare_proxy
    conn.connect()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 721, in connect
    self._tunnel()
  File "c:\Users\JIJUNDU\AppDatal\anaconda3\lib\site-packages\urllib3\connection.py", line 259, in _tunnel
    raise OSError(f"Tunnel connection failed: {code} {message.strip()}")
OSError: Tunnel connection failed: 403 Forbidden

The above exception was the direct cause of the following exception:

urllib3.exceptions.ProxyError: ('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden'))

The above exception was the direct cause of the following exception:





# Agent: Editor
## Final Answer: 
```markdown
# Best Practices for Using Crew AI to Write Patents

## Introduction

In the fast-paced world of intellectual property, the emergence of artificial intelligence (AI) tools is revolutionizing how patents are written. One standout entity in this realm is Crew AI, an innovative platform that empowers inventors, patent attorneys, and businesses to streamline their patent drafting processes. Understanding how to leverage Crew AI effectively can save time, reduce frustration, and improve the overall quality of patent applications. This article aims to delve into the best practices for using Crew AI in patent writing, ultimately aiding professionals in navigating the complexities of patent documentation.

The relevance of this topic cannot be overstated, given the growing reliance on AI technologies in legal processes. With platforms like Crew AI, the opportunity to enhance efficiency and accuracy in patent writing has never been clearer. As inn

In [23]:
Markdown(result)

# Best Practices for Using Crew AI to Write Patents

## Introduction

Patents play a crucial role in protecting intellectual property rights and providing legal security for inventors and businesses. In today's digital era, the process of patent writing has been significantly transformed by technological advancements. One such innovation is Crew AI, a state-of-the-art tool that leverages artificial intelligence to streamline and enhance the patent drafting process.

## Latest Trends and Key Players

Recent advancements in AI technology have greatly improved the efficiency and accuracy of patent writing. Leading companies like Crew AI have emerged as key players in this space, offering cutting-edge solutions to patent attorneys, inventors, and businesses. The integration of AI in patent writing has been a major topic of discussion, with frequent updates highlighting its impact on the industry.

## Target Audience Analysis

The primary audience for Crew AI includes patent attorneys, inventors, and businesses looking to protect their intellectual property rights. These individuals are interested in simplifying the patent writing process, increasing efficiency, and ensuring the accuracy of their patent documentation. Common challenges they face include manual drafting errors and the high costs associated with traditional methods of patent writing.

## Best Practices for Using Crew AI to Write Patents

To effectively harness the power of Crew AI for patent writing, it is essential to follow these best practices:
1. Gain a thorough understanding of the platform and its functionalities.
2. Provide accurate information to ensure precision in patent drafting.
3. Thoroughly review and make necessary edits to the AI-generated content for completeness and accuracy.
4. Incorporate relevant keywords and phrases to optimize the content for search engines.
5. Utilize case studies and success stories to grasp the benefits of AI in the patent writing process.

## Call to Action

Experience the advantages of using Crew AI for patent writing by taking advantage of a free trial or discount offer today. Explore the capabilities of the tool firsthand and reach out to our team for personalized guidance on maximizing your patent drafting process with Crew AI. Embrace AI technology to stay ahead of the curve in your intellectual property endeavors and enhance efficiency and accuracy in patent drafting.

In conclusion, implementing Crew AI for patent writing presents a groundbreaking solution for improving efficiency and accuracy in the drafting process. By embracing AI technology, you can elevate your intellectual property endeavors and stay ahead in the ever-evolving landscape of patent writing.

---
SEO Keywords: AI patent writing, Crew AI for patents, Patent drafting best practices, Using AI for patent writing, Patent writing tools

Resources:
1. "The Future of Patent Writing: AI Technology and Its Impact" - Forbes
2. "How AI is Revolutionizing Patent Drafting" - Harvard Business Review
3. Crew AI website and customer testimonials

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).